In [1]:
from __future__ import print_function

import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.constraints import nonneg
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
from sklearn.utils.extmath import softmax
import tensorflow as tf
from numba import jit
import time
import math

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 20

In [3]:
# train the model
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


60000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
# model.add(Dense(256, use_bias=0, activation='relu', input_shape=(784,)))
# model.add(Dense(256, use_bias=0, activation='relu'))
# model.add(Dense(256, use_bias=0, activation='relu'))

model.add(Dense(256, use_bias=0, activation='relu', input_shape=(784,)))
model.add(Dense(256, use_bias=0, activation='relu'))
model.add(Dense(256, use_bias=0, activation='relu'))

model.add(Dense(num_classes, use_bias=0, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               200704    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65536     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65536     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2560      
Total params: 334,336
Trainable params: 334,336
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s 71us/step - loss: 0.2548 - acc: 0.9221 - val_loss: 0.1235 - val_acc: 0.9616
Epoch 2/20
60000/60000 [==============================] - 2s 29us/step - loss: 0.0969 - acc: 0.9704 - val_loss: 0.1066 - val_acc: 0.9652
Epoch 3/20
60000/60000 [==============================] - 2s 30us/step - loss: 0.0640 - acc: 0.9798 - val_loss: 0.0711 - val_acc: 0.9798
Epoch 4/20
60000/60000 [==============================] - 2s 31us/step - loss: 0.0468 - acc: 0.9858 - val_loss: 0.0921 - val_acc: 0.9772
Epoch 5/20
60000/60000 [==============================] - 2s 30us/step - loss: 0.0357 - acc: 0.9888 - val_loss: 0.0883 - val_acc: 0.9762
Epoch 6/20
60000/60000 [==============================] - 2s 29us/step - loss: 0.0301 - acc: 0.9908 - val_loss: 0.0889 - val_acc: 0.9785
Epoch 7/20
60000/60000 [==============================] - 2s 30us/step - loss: 0.0251 - acc: 0.9926 - val_loss: 0.0951 - val_acc

In [16]:
model.save_weights('my_model_weights.h5')

In [17]:
model2 = Sequential()
model2.add(Dense(256, use_bias=0, activation='relu', input_shape=(784,)))
#model.add(Dropout(0.2))
model2.add(Dense(256, use_bias=0, activation='relu'))
model2.add(Dense(256, use_bias=0, activation='relu'))
#model.add(Dropout(0.2))
model2.add(Dense(num_classes, use_bias=0, activation='softmax'))

model2.load_weights('my_model_weights.h5')


In [36]:
# functions for doing inference in a for loop


@jit
def quantize_arr4(arr, max_val):
    """Quantization based on linear rescaling over min/max range.
    """
    quantized = np.round(4 * (arr/max_val))
    quantized /= 4
    quantized *= max_val
    return quantized
@jit
def feedforward(inputs,weights,absmax,activate_relu=1,leaf=64,spread_branch=1/256):#inputs.shape[0]=784
    inputs = inputs * np.random.uniform(1-tl_variation(),1+tl_variation(),inputs.shape)
    
    copy_num = weights[0].shape[1]
    spliter_count = math.log2(leaf) + 3

    random_input = np.repeat(inputs,copy_num,0).reshape(inputs.shape[0],copy_num) * spliter_forest(inputs.shape[0],copy_num)

    
    random_input *= coupling_loss()    
       
    weights_random = weights[0] * np.random.uniform(0.95,1.05,weights[0].shape)
    
    result = random_input * weights_random * propagation_loss() * splitter_loss()
   
    random_input[np.abs(random_input)<1e-3]=0
    
    result = result * 1/2 * splitter_loss()
    
    result[np.abs(result)<2e-7]=0
    
    result = np.sum (result,0) 
    
    output = amplifier(result , 2 * absmax * leaf /(1 - spread_branch) / coupling_loss()/\
                       splitter_loss() ** spliter_count/propagation_loss() ** (spliter_count+1)) * propagation_loss() 
    
    if activate_relu==1:
        output=relu(output)
        output[output>1]=1
        
    return output

@jit
def relu(output):
  nextinput = np.zeros((output.shape[0]))
  for i in range(nextinput.shape[0]):
    nextinput[i] = max(0,output[i])
  return nextinput

@jit
def normalize(array,quanti=8):#normalize and quantize the weight, denormalize in feedfoward
    w=np.asarray(array)
    absmax=np.amax(np.abs(w))
    w /= absmax
    quantScale=1/quanti/2
    for i in range(-quanti,quanti):
        w[np.abs(w-i/quanti)<=quantScale] = i/quanti
    w[np.abs(w-quantScale/2)<quantScale/2]=1/quanti
    w[np.abs(w-(1-quantScale/2))<quantScale/2]=(quanti-1)/quanti
    return w
                       

@jit
def network(x_test, y_test):#x_test is 1*784, one 28*28 image
    weights0=np.asarray(model2.layers[0].get_weights())
    absmax=np.amax(np.abs(weights0))
    o1 = feedforward(x_test,normalize(weights0),absmax)
    weights1=np.asarray(model2.layers[1].get_weights())
    absmax=np.amax(np.abs(weights1))
    o1 = feedforward(o1,normalize(weights1),absmax)
    weights2=np.asarray(model2.layers[2].get_weights())
    absmax=np.amax(np.abs(weights2))
    o1 = feedforward(o1,normalize(weights2),absmax)
    weights3=np.asarray(model2.layers[3].get_weights())
    absmax=np.amax(np.abs(weights3))
    o1 = feedforward(o1,normalize(weights3),absmax,0)
    pred_out = np.zeros((1,o1.shape[0]))
    pred_out[0] = o1
    correct_out = np.zeros((1,y_test.shape[0]))
    correct_out[0] = y_test
    return (np.argmax(pred_out) == np.argmax(correct_out))
@jit
def spliter_tree(root_value=1,leaf=64,variation=0.05):
    height=math.ceil(math.log2(leaf))
    arr=[]
    for i in range(0,height+1):
        arr.append([])
        node=2**i
        if i==0:
            arr[i].append(root_value)
        else:
            for j in range(0,int(node/2)):
                    v=np.random.uniform(1-variation,1+variation)#inject splitter loss here
                    arr[i].append(v*(1/2)*arr[i-1][math.floor(j)]*propagation_loss()*splitter_loss())
                    arr[i].append((2-v)*(1/2)*arr[i-1][math.floor(j)]*propagation_loss()*splitter_loss())
    return np.asarray(arr[-1])
@jit
def spliter_grove(total=256,leaf=64,spread_branch=1/256,variation=0.05):
    tree_num=math.ceil(total/leaf)
    arr=np.zeros((tree_num,leaf))
    v=np.random.uniform(1-variation,1+variation,tree_num)
    right=np.zeros(tree_num)
    left=np.zeros(tree_num)
    right[0]=v[0]*spread_branch
    left[0]=1-right[0]
    arr[0]=spliter_tree(left[0]*propagation_loss()*splitter_loss(),leaf,variation)
    for i in range(1,tree_num):
        right[i]=amplifier(right[i-1]*splitter_loss()*propagation_loss(),1/(spread_branch*splitter_loss()*propagation_loss()))\
        *v[i]*spread_branch
        left[i]=1-right[i]
        arr[i]=spliter_tree(left[i]*propagation_loss()*splitter_loss(),leaf,variation)
    return arr.flatten()[:total] 
@jit
def spliter_forest(input_num=784,grove_len=256,leaf=64):
    arr=np.zeros((input_num,grove_len))
    for i in range(0,input_num):
        arr[i]=spliter_grove(grove_len)
    return arr 
def amplifier(x, multiple=64, amp_vari=0.05):
    x = x / 6.25
    delta_in= x * np.random.uniform(-amp_vari,amp_vari)
    multiple *= 6.25
    x_out = x * multiple + multiple * delta_in*10**(1/2) # x * multiple or x_in * multiple ?
    return x_out
def propagation_loss(length=15*10e-4,loss_per_length=0.3):
    return 1/10**(length*loss_per_length/10)
def splitter_loss(loss=0.2):
    return 1/10**(loss/10)
def coupling_loss(layers=2):
    return 1/10**(layers*0.5/10)
def tl_variation(vari=0.05):
    return vari


In [ ]:
@jit
def main(logname='newest.txt'):
    log=open(logname,'w')
    log.write(time.strftime("%Y-%m-%d %H:%M:%S\n", time.localtime()))
    log.close()
    ave=np.ones(100)
    (x_train1, y_train1), (x_test1, y_test1) = mnist.load_data()
    x_test1 = x_test1.reshape(10000, 784)
    x_test1 = x_test1.astype('float32')
    x_test1 /= np.amax(x_test1)
    x_test1 = quantize_arr4(x_test1,np.amax(x_test1))
    y_test1 = keras.utils.to_categorical(y_test1, num_classes)
    total_test = x_test1.shape[0]
    for j in range(0,100):
        correct_test = 0        
        for i in range(x_test1.shape[0]):
          if (network(x_test1[i],y_test1[i])):
             correct_test += 1
          if (i+1)%10==0: # to test the function
             print ('inference %d current accuracy at %i:'%(j+1,i+1),correct_test/(i+1))
        print ('The accuracy of inference %d: %f'%(j+1,correct_test/float(total_test)))
        ave[j]=correct_test/float(total_test)
        log=open(logname,'a')
        log.write('The accuracy of inference %d: %f;\n'%(j+1,correct_test/float(total_test)))
        log.close()
    log=open(logname,'a')
    log.write('The mean of accuracy is %f.\n'%np.average(ave))
    log.write('The variance of accuracy is %f.\n'%np.var(ave))
    log.close()
main('12_2.txt')

inference 1 current accuracy at 10: 1.0
inference 1 current accuracy at 20: 0.95
inference 1 current accuracy at 30: 0.9666666666666667
inference 1 current accuracy at 40: 0.975
inference 1 current accuracy at 50: 0.98
inference 1 current accuracy at 60: 0.9833333333333333
inference 1 current accuracy at 70: 0.9857142857142858
inference 1 current accuracy at 80: 0.9875
inference 1 current accuracy at 90: 0.9888888888888889
inference 1 current accuracy at 100: 0.99
inference 1 current accuracy at 110: 0.9818181818181818
inference 1 current accuracy at 120: 0.9833333333333333
inference 1 current accuracy at 130: 0.9846153846153847
inference 1 current accuracy at 140: 0.9857142857142858
inference 1 current accuracy at 150: 0.9866666666666667
inference 1 current accuracy at 160: 0.975
inference 1 current accuracy at 170: 0.9764705882352941
inference 1 current accuracy at 180: 0.9722222222222222
inference 1 current accuracy at 190: 0.968421052631579
inference 1 current accuracy at 200: 0.97

inference 1 current accuracy at 1520: 0.9539473684210527
inference 1 current accuracy at 1530: 0.9529411764705882
inference 1 current accuracy at 1540: 0.9525974025974026
inference 1 current accuracy at 1550: 0.952258064516129
inference 1 current accuracy at 1560: 0.9525641025641025
inference 1 current accuracy at 1570: 0.9522292993630573
inference 1 current accuracy at 1580: 0.9525316455696202
inference 1 current accuracy at 1590: 0.9528301886792453
inference 1 current accuracy at 1600: 0.953125
inference 1 current accuracy at 1610: 0.953416149068323
inference 1 current accuracy at 1620: 0.9537037037037037
inference 1 current accuracy at 1630: 0.9539877300613497
inference 1 current accuracy at 1640: 0.9536585365853658
inference 1 current accuracy at 1650: 0.953939393939394
inference 1 current accuracy at 1660: 0.9542168674698795
inference 1 current accuracy at 1670: 0.9544910179640719
inference 1 current accuracy at 1680: 0.9535714285714286
inference 1 current accuracy at 1690: 0.9526

inference 1 current accuracy at 2970: 0.9542087542087542
inference 1 current accuracy at 2980: 0.9540268456375839
inference 1 current accuracy at 2990: 0.954180602006689
inference 1 current accuracy at 3000: 0.954
inference 1 current accuracy at 3010: 0.9541528239202658
inference 1 current accuracy at 3020: 0.9543046357615894
inference 1 current accuracy at 3030: 0.9541254125412542
inference 1 current accuracy at 3040: 0.9542763157894737
inference 1 current accuracy at 3050: 0.9544262295081967
inference 1 current accuracy at 3060: 0.9545751633986929
inference 1 current accuracy at 3070: 0.9543973941368078
inference 1 current accuracy at 3080: 0.9545454545454546
inference 1 current accuracy at 3090: 0.9546925566343042
inference 1 current accuracy at 3100: 0.9548387096774194
inference 1 current accuracy at 3110: 0.954983922829582
inference 1 current accuracy at 3120: 0.9548076923076924
inference 1 current accuracy at 3130: 0.9549520766773163
inference 1 current accuracy at 3140: 0.954777

inference 1 current accuracy at 4420: 0.9518099547511312
inference 1 current accuracy at 4430: 0.9516930022573363
inference 1 current accuracy at 4440: 0.9518018018018019
inference 1 current accuracy at 4450: 0.9519101123595506
inference 1 current accuracy at 4460: 0.9520179372197309
inference 1 current accuracy at 4470: 0.9521252796420582
inference 1 current accuracy at 4480: 0.9520089285714286
inference 1 current accuracy at 4490: 0.9521158129175946
inference 1 current accuracy at 4500: 0.952
inference 1 current accuracy at 4510: 0.9518847006651885
inference 1 current accuracy at 4520: 0.9519911504424778
inference 1 current accuracy at 4530: 0.9520971302428256
inference 1 current accuracy at 4540: 0.9519823788546256
inference 1 current accuracy at 4550: 0.9520879120879121
inference 1 current accuracy at 4560: 0.9521929824561404
inference 1 current accuracy at 4570: 0.9520787746170678
inference 1 current accuracy at 4580: 0.9519650655021834
inference 1 current accuracy at 4590: 0.9520

inference 1 current accuracy at 5870: 0.9562180579216354
inference 1 current accuracy at 5880: 0.9562925170068027
inference 1 current accuracy at 5890: 0.9561969439728353
inference 1 current accuracy at 5900: 0.9561016949152542
inference 1 current accuracy at 5910: 0.956175972927242
inference 1 current accuracy at 5920: 0.9559121621621621
inference 1 current accuracy at 5930: 0.9558178752107925
inference 1 current accuracy at 5940: 0.9555555555555556
inference 1 current accuracy at 5950: 0.9556302521008403
inference 1 current accuracy at 5960: 0.9557046979865772
inference 1 current accuracy at 5970: 0.9556113902847572
inference 1 current accuracy at 5980: 0.9555183946488295
inference 1 current accuracy at 5990: 0.9550918196994992
inference 1 current accuracy at 6000: 0.9551666666666667
inference 1 current accuracy at 6010: 0.9550748752079867
inference 1 current accuracy at 6020: 0.954983388704319
inference 1 current accuracy at 6030: 0.954726368159204
inference 1 current accuracy at 60

...

inference 4 current accuracy at 2210: 0.9479638009049773
inference 4 current accuracy at 2220: 0.9481981981981982
inference 4 current accuracy at 2230: 0.9484304932735426
inference 4 current accuracy at 2240: 0.9486607142857143
inference 4 current accuracy at 2250: 0.9488888888888889
inference 4 current accuracy at 2260: 0.9491150442477876
inference 4 current accuracy at 2270: 0.9493392070484582
inference 4 current accuracy at 2280: 0.9495614035087719
inference 4 current accuracy at 2290: 0.9497816593886463
inference 4 current accuracy at 2300: 0.9495652173913044
inference 4 current accuracy at 2310: 0.9497835497835497
inference 4 current accuracy at 2320: 0.95
inference 4 current accuracy at 2330: 0.9493562231759657
inference 4 current accuracy at 2340: 0.9495726495726495
inference 4 current accuracy at 2350: 0.9497872340425532
inference 4 current accuracy at 2360: 0.95
inference 4 current accuracy at 2370: 0.950210970464135
inference 4 current accuracy at 2380: 0.9504201680672268
inf

inference 4 current accuracy at 3670: 0.9517711171662125
inference 4 current accuracy at 3680: 0.9519021739130434
inference 4 current accuracy at 3690: 0.9520325203252032
inference 4 current accuracy at 3700: 0.9521621621621622
inference 4 current accuracy at 3710: 0.9522911051212938
inference 4 current accuracy at 3720: 0.9524193548387097
inference 4 current accuracy at 3730: 0.9522788203753351
inference 4 current accuracy at 3740: 0.9521390374331551
inference 4 current accuracy at 3750: 0.952
inference 4 current accuracy at 3760: 0.9518617021276595
inference 4 current accuracy at 3770: 0.9514588859416445
inference 4 current accuracy at 3780: 0.951058201058201
inference 4 current accuracy at 3790: 0.9511873350923483
inference 4 current accuracy at 3800: 0.9513157894736842
inference 4 current accuracy at 3810: 0.9511811023622048
inference 4 current accuracy at 3820: 0.9507853403141361
inference 4 current accuracy at 3830: 0.9509138381201044
inference 4 current accuracy at 3840: 0.95078

inference 4 current accuracy at 5140: 0.9503891050583657
inference 4 current accuracy at 5150: 0.9500970873786407
inference 4 current accuracy at 5160: 0.9501937984496124
inference 4 current accuracy at 5170: 0.9502901353965184
inference 4 current accuracy at 5180: 0.9503861003861004
inference 4 current accuracy at 5190: 0.9504816955684008
inference 4 current accuracy at 5200: 0.9505769230769231
inference 4 current accuracy at 5210: 0.9506717850287908
inference 4 current accuracy at 5220: 0.9505747126436782
inference 4 current accuracy at 5230: 0.9504780114722753
inference 4 current accuracy at 5240: 0.9505725190839694
inference 4 current accuracy at 5250: 0.9506666666666667
inference 4 current accuracy at 5260: 0.9507604562737643
inference 4 current accuracy at 5270: 0.9506641366223909
inference 4 current accuracy at 5280: 0.9507575757575758
inference 4 current accuracy at 5290: 0.9508506616257089
inference 4 current accuracy at 5300: 0.9509433962264151
inference 4 current accuracy at

inference 4 current accuracy at 6590: 0.9541729893778452
inference 4 current accuracy at 6600: 0.953939393939394
inference 4 current accuracy at 6610: 0.9540090771558245
inference 4 current accuracy at 6620: 0.9540785498489426
inference 4 current accuracy at 6630: 0.9538461538461539
inference 4 current accuracy at 6640: 0.9539156626506025
inference 4 current accuracy at 6650: 0.9538345864661654
inference 4 current accuracy at 6660: 0.9537537537537537
inference 4 current accuracy at 6670: 0.9538230884557721
inference 4 current accuracy at 6680: 0.9538922155688623
inference 4 current accuracy at 6690: 0.9539611360239163
inference 4 current accuracy at 6700: 0.9540298507462687
inference 4 current accuracy at 6710: 0.9539493293591654
inference 4 current accuracy at 6720: 0.9540178571428571
inference 4 current accuracy at 6730: 0.9540861812778604
inference 4 current accuracy at 6740: 0.9541543026706232
inference 4 current accuracy at 6750: 0.9542222222222222
inference 4 current accuracy at 

inference 4 current accuracy at 8040: 0.9580845771144278
inference 4 current accuracy at 8050: 0.958136645962733
inference 4 current accuracy at 8060: 0.9580645161290322
inference 4 current accuracy at 8070: 0.9581164807930607
inference 4 current accuracy at 8080: 0.9581683168316831
inference 4 current accuracy at 8090: 0.9582200247218788
inference 4 current accuracy at 8100: 0.9582716049382716
inference 4 current accuracy at 8110: 0.9583230579531443
inference 4 current accuracy at 8120: 0.9582512315270936
inference 4 current accuracy at 8130: 0.9583025830258303
inference 4 current accuracy at 8140: 0.9583538083538083
inference 4 current accuracy at 8150: 0.9582822085889571
inference 4 current accuracy at 8160: 0.9583333333333334
inference 4 current accuracy at 8170: 0.9583843329253366
inference 4 current accuracy at 8180: 0.9584352078239609
inference 4 current accuracy at 8190: 0.9583638583638584
inference 4 current accuracy at 8200: 0.9582926829268292
inference 4 current accuracy at 